In [1]:
import pandas as pd
import seaborn as sns
import openpyxl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

In [2]:
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna

In [3]:
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import RidgeCV, Ridge 
from sklearn.svm import LinearSVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [4]:
from Functions.helper_functions import * 

In [5]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None  # default='warn'

# Data import

In [6]:
# train_wp1 = pd.read_csv('Data/Preprocessing/WP1_train_preprocessed.csv', sep=',')
# train_wp2 = pd.read_csv('Data/Preprocessing/WP2_train_preprocessed.csv', sep=',')
# train_wp3 = pd.read_csv('Data/Preprocessing/WP3_train_preprocessed.csv', sep=',')
# train_wp4 = pd.read_csv('Data/Preprocessing/WP4_train_preprocessed.csv', sep=',')
# train_wp5 = pd.read_csv('Data/Preprocessing/WP5_train_preprocessed.csv', sep=',')
# train_wp6 = pd.read_csv('Data/Preprocessing/WP6_train_preprocessed.csv', sep=',')

# test_wp1 = pd.read_csv('Data/Preprocessing/WP1_test_preprocessed.csv', sep=',')
# test_wp2 = pd.read_csv('Data/Preprocessing/WP2_test_preprocessed.csv', sep=',')
# test_wp3 = pd.read_csv('Data/Preprocessing/WP3_test_preprocessed.csv', sep=',')
# test_wp4 = pd.read_csv('Data/Preprocessing/WP4_test_preprocessed.csv', sep=',')
# test_wp5 = pd.read_csv('Data/Preprocessing/WP5_test_preprocessed.csv', sep=',')
# test_wp6 = pd.read_csv('Data/Preprocessing/WP6_test_preprocessed.csv', sep=',')
# test_dates = pd.read_csv('Data/Initial/test.csv', sep=',').date.values

In [7]:
to_drop = ['date','wd','forecast_time', 'forecast', "forecast_dist"]

# Voting from previous submissions

In [48]:
pred_stack = pd.read_csv('Predictions/submission_nb_16_stacking_0-2xgboost+2lgbm_1-LinearSVR.csv', sep=';')
pred_dnn =  pd.read_csv('Predictions/submission_nb_6_full_dnn.csv', sep=';')
pred_lgbm_maxabs = pd.read_csv('Predictions/submission_nb_9_full_maxabs-lgbm.csv', sep=';')
pred_xgboost = pd.read_csv('Predictions/submission_nb_8_full_absmax-xgboost.csv', sep=';')
pred_ridge = pd.read_csv('Predictions/submission_nb_6_full_ridgecv.csv', sep=';')
pred_lgbm_std = pd.read_csv('Predictions/submission_nb_7_full_std-lgbm.csv', sep=';')


score_1 = 0.112119
pred_1 = pd.read_csv('Predictions/submission_nb_31_blending_stacking_0-GBoost-RidgeCV-LinearSVR-ExtraTrees-KRR-ENet_1-LinearSVR.csv', sep=';')
w_1 = 1

score_2 = 0.112449
pred_2 = pd.read_csv('Predictions/submission_nb_28_metalearner_stacking_0-2gboost-2lgbm-ridge-linearsvr-extratrees_1-Ridgecv.csv', sep=';')
w_2 = 1

score_3 = 0.115305
pred_3 = pd.read_csv('Predictions/submission_nb_16_stacking_0-2xgboost+2lgbm_1-LinearSVR.csv', sep=';')
w_3 = 1

score_4 = 0.115536
pred_4 = pd.read_csv('Predictions/submission_nb_32_blending_stacking_0-10-LGBM_1-RidgeCV.csv', sep=';')
w_4 = 1

score_5 = 0.116202
pred_5 = pd.read_csv('Predictions/submission_nb_15_stacking_0-xgboost+lgbm_1-LinearSVR.csv', sep=';')
w_5 = 1

score_6 = 0.116523
pred_6 = pd.read_csv('Predictions/submission_nb_33_blending_stacking_0-10-XGBOOST_1-RidgeCV.csv', sep=';')
w_6 = 1

score_7 = 0.118056
pred_7 = pd.read_csv('Predictions/submission_nb_25_cv_stacking_0-ENet-GBoost-KRR-SVR-ridge-xtratree_1-Lasso.csv', sep=';')
w_7 = 1


score_8 = 0.119004
pred_8 = pd.read_csv('Predictions/submission_nb_24_stacking_0-2gboost+2lgbm+ridge+linearsvr+extratrees_1-Ridgecv.csv', sep=';')
w_8 = 1

sum_weights = w_1 + w_2 + w_3 + w_4 + w_5 + w_6 + w_7 + w_8

In [49]:
sum_weights

8

# Voting method 

## Simple average

In [50]:
pred_av = (w_1*pred_1 + w_2*pred_2 + w_3*pred_3 + w_4*pred_4 + w_5*pred_5 + w_6*pred_6 + w_7*pred_7 + + w_8*pred_8)/sum_weights

In [51]:
pred_av["date"] = pred_av["date"].astype(int)
pred_av

,date,wp1,wp2,wp3,wp4,wp5,wp6
0,2011010101,0.689188,0.329246,0.061754,0.540545,0.728689,0.596446
1,2011010102,0.633314,0.355099,0.033871,0.485653,0.725137,0.609132
2,2011010103,0.589158,0.389185,0.090877,0.487014,0.732418,0.617269
3,2011010104,0.612945,0.378562,0.170428,0.526409,0.718101,0.630358
4,2011010105,0.684021,0.293866,0.308869,0.604287,0.755498,0.632843
...,...,...,...,...,...,...,...
7435,2012062420,0.112774,0.045312,0.255537,0.119175,0.207148,0.184635
7436,2012062421,0.104274,0.035682,0.338672,0.115746,0.260539,0.212569
7437,2012062422,0.075431,0.033170,0.425077,0.090197,0.212895,0.193319
7438,2012062423,0.056973,0.030595,0.470779,0.046954,0.142213,0.165646


In [52]:
nb_sub = 35
model = "votingreg"
# models = "dnn-stdlgbm-maxabslgbm-xgboost-ridge-stack"
models = "8-best_models"
weight = 'Weighted'

pred_av.to_csv(f'Predictions/submission_nb_{nb_sub}_{model}_{models}.csv', index=False, sep=';')
f = open(f"Predictions/submission-{nb_sub}_{model}_{models}.txt", "x")
f.write(f"Model: Predictions/submission_nb_31_blending_stacking_0-GBoost-RidgeCV-LinearSVR-ExtraTrees-KRR-ENet_1-LinearSVR.csv\n")
f.write(f"Model weight: {w_1}\n\n")
f.write(f"Model: Predictions/submission_nb_28_metalearner_stacking_0-2gboost-2lgbm-ridge-linearsvr-extratrees_1-Ridgecv.csv\n")
f.write(f"Model weight: {w_2}\n\n")
f.write(f"Model: Predictions/submission_nb_16_stacking_0-2xgboost+2lgbm_1-LinearSVR.csv\n")
f.write(f"Model weight: {w_3}\n\n")
f.write(f"Model: Predictions/submission_nb_32_blending_stacking_0-10-LGBM_1-RidgeCV.csv\n")
f.write(f"Model weight: {w_4}\n\n")
f.write(f"Model: Predictions/submission_nb_15_stacking_0-xgboost+lgbm_1-LinearSVR.csv\n")
f.write(f"Model weight: {w_5}\n\n")
f.write(f"Model: Predictions/submission_nb_33_blending_stacking_0-10-XGBOOST_1-RidgeCV.csv\n")
f.write(f"Model weight: {w_6}\n\n")
f.write(f"Model: Predictions/submission_nb_25_cv_stacking_0-ENet-GBoost-KRR-SVR-ridge-xtratree_1-Lasso.csv\n")
f.write(f"Model weight: {w_7}\n\n")
f.write(f"Model: Predictions/Predictions/submission_nb_24_stacking_0-2gboost+2lgbm+ridge+linearsvr+extratrees_1-Ridgecv.csv\n")
f.write(f"Model weight: {w_8}\n\n")

f.write(f"Type of weighted : {weight}")
f.close()